Run this to mount your personal google drive.
Ensure you have COCO-Human-Pose repo cloned into your drive at the path on line 6.

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# Switch directories
%cd /content/gdrive/MyDrive/Colab\ Data/COCO-Human-Pose
%ls

Mounted at /content/gdrive
/content/gdrive/MyDrive/Colab Data/COCO-Human-Pose
coco_df.py               hourglass_blocks.py  output/           submodules/
constants.py             hourglass.py         __pycache__/      test.py
data/                    LICENSE              README.md         train.py
data_generator.py        logs/                report/
data_generator_tests.py  models/              requirements.txt
evaluation.py            notebooks/           scripts/


In [ ]:
!ls /content/gdrive/MyDrive/

Run this to load coco train/val keypoint annotations into an in-mem dataframe which can be queried. Ensure the paths make sense. It also this to declares a function that displays Image w/ annotations by file name

In [16]:
# Filename is a globally unique identifier among all train/val/test splits
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import coco_df
from pycocotools.coco import COCO
from matplotlib.patches import Rectangle
pylab.rcParams['figure.figsize'] = (30.0, 30.0)

train_annot_path = '/content/gdrive/MyDrive/COCO/2017/annotations/person_keypoints_train2017.json'
val_annot_path = '/content/gdrive/MyDrive/COCO/2017/annotations/person_keypoints_val2017.json'
train_coco = COCO(train_annot_path) # load annotations for training set
val_coco = COCO(val_annot_path) # load annotations for validation set

df = coco_df.get_df(train_annot_path,val_annot_path)

def display_img(image_file_name):
  # Determine if img exists and if it is in train or val set
  img_df_rows = df.loc[df['path'] == 'train2017/'+image_file_name]
  if len(img_df_rows) == 0:
    img_df_rows = df.loc[df['path'] == 'val2017/'+image_file_name]
    if len(img_df_rows) == 0:
      print("Image with filename: " + image_file_name + " does not exist.")
      return
  
  coco = train_coco if img_df_rows['source'].iloc[0] == 0 else val_coco

  # Get img id from file name
  imgId = img_df_rows['src_set_image_id'].iloc[0]
  img = coco.imgs[imgId]
  I = io.imread(img['coco_url']) # load image from URL (no need to store image locally)

  # load and display keypoints annotations
  plt.subplot(1,2,1)
  plt.imshow(I)
  plt.axis('off')

  plt.subplot(1,2,2)
  plt.imshow(I)
  annIds = coco.getAnnIds(imgIds=[imgId])
  anns = coco.loadAnns(annIds)
  bbox= anns[0]['bbox']
  plt.gca().add_patch(Rectangle((bbox[0],bbox[1]),bbox[2],bbox[3],
                  edgecolor='red',
                  facecolor='none',
                  lw=4))
  coco.showAnns(anns)
  plt.show()

loading annotations into memory...
Done (t=8.84s)
creating index...
index created!
loading annotations into memory...
Done (t=2.78s)
creating index...
index created!
loading annotations into memory...
Done (t=8.72s)
creating index...
index created!
loading annotations into memory...
Done (t=0.28s)
creating index...
index created!


Example of using the display_img function

In [ ]:
image_file_name = '000000163171.jpg'
display_img(image_file_name)

In [17]:
# see the contents of the coco-df
df.loc[df['is_crowd'] == 0]

,src_set_image_id,coco_url,path,width,height,ann_id,is_crowd,bbox,bbox_area,area,num_keypoints,keypoints,segmentation,source
0,36,http://images.cocodataset.org/train2017/000000...,train2017/000000000036.jpg,481,640,453991,0,"[167.58, 162.89, 310.61, 465.19]",144492.6659,86145.29710,13,"[250, 244, 2, 265, 223, 2, 235, 235, 2, 309, 2...","[[345.28, 220.68, 348.17, 269.8, 355.4, 307.36...",0
1,49,http://images.cocodataset.org/train2017/000000...,train2017/000000000049.jpg,381,500,198565,0,"[203.39, 260.43, 65.69, 73.04]",4797.9976,1754.81755,13,"[251, 276, 2, 253, 275, 2, 250, 274, 2, 259, 2...","[[260.74, 281.51, 263.68, 283.47, 263.68, 290....",0
2,49,http://images.cocodataset.org/train2017/000000...,train2017/000000000049.jpg,381,500,254537,0,"[118.43, 261.32, 56.91, 62.93]",3581.3463,1349.98230,11,"[158, 270, 1, 160, 269, 2, 157, 269, 1, 167, 2...","[[153.35, 277.78, 156.08, 275.73, 158.81, 270....",0
3,49,http://images.cocodataset.org/train2017/000000...,train2017/000000000049.jpg,381,500,1211660,0,"[119.34, 334.21, 11.94, 33.57]",400.8258,250.81990,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[125.66, 367.36, 127.77, 367.36, 128.75, 367....",0
4,49,http://images.cocodataset.org/train2017/000000...,train2017/000000000049.jpg,381,500,2007474,0,"[284.34, 333.4, 10.88, 27.43]",298.4384,162.90005,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[289.7, 357.59, 291.97, 358.89, 293.43, 360.8...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273464,581357,http://images.cocodataset.org/val2017/00000058...,val2017/000000581357.jpg,612,612,1691804,0,"[180.89, 440.51, 21.1, 38.36]",809.3960,371.49395,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[184, 440.75, 180.89, 457.77, 184.72, 463.53,...",1
273465,581357,http://images.cocodataset.org/val2017/00000058...,val2017/000000581357.jpg,612,612,1698852,0,"[249.22, 419.65, 13.2, 24.53]",323.7960,169.55635,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[251.96, 444.18, 249.22, 442.46, 250.42, 438....",1
273466,581357,http://images.cocodataset.org/val2017/00000058...,val2017/000000581357.jpg,612,612,1726027,0,"[211.18, 420.26, 23.38, 40.03]",935.9014,542.16740,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[212.78, 430.08, 211.18, 435.02, 212.49, 435....",1
273467,581357,http://images.cocodataset.org/val2017/00000058...,val2017/000000581357.jpg,612,612,1748315,0,"[230.28, 420.86, 19.02, 37.35]",710.3970,371.93055,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[242.81, 427.82, 245.12, 436.17, 246.05, 440....",1


# All code blocks below are example queries on the coco df.

In [ ]:
# Number of images in train + val
len(coco_df.groupby('path',as_index=False).size())

In [ ]:
# Breakdown of num images with a given num annotations
coco_df.groupby('path',as_index=False).size().rename(columns = {'size': 'num_annotations'}).groupby('num_annotations',as_index=False).size().rename(columns={'size':'num_images'}).sort_values(by='num_images',ascending=False)

In [ ]:
# images with crowds
coco_df.loc[coco_df['is_crowd'] == 1].groupby('path', as_index=False).size()

In [ ]:
# Breakdown of num of annotations with a given num keypoints
coco_df.groupby('num_keypoints',as_index=False).size().sort_values(by='size',ascending=False)

In [ ]:
# Annotations with X keypoints
coco_df.loc[coco_df['num_keypoints'] == 0]

In [ ]:
# The image with the most people (20)
coco_df.groupby('path',as_index=False).size().loc[coco_df.groupby('path',as_index=False).size()['size'] == 20]

In [ ]:
# how many images have an annotation with x or more keypoints?
# how many of those images have only one annotation?

# num images where best annotation has X keypoints
df = coco_df.groupby('path',as_index=False).max('num_keypoints').groupby('num_keypoints',as_index=False).size()

# increase sizes so that sum represents all images with X or MORE keypoints
for i, row in df.iterrows():
  row['size'] += sum(df.loc[df['num_keypoints'] > row['num_keypoints']]['size'])

df.rename(columns={'size':'num_imgs'},inplace=True)

anns_per_img = coco_df.groupby('path',as_index=False).size()
imgs_w_one_ann = anns_per_img.loc[anns_per_img['size'] == 1]
imgs_w_one_ann = pd.merge(imgs_w_one_ann,coco_df)
df2 = imgs_w_one_ann.groupby('num_keypoints',as_index=False).size()

# increase sizes so that sum represents all images with X or MORE keypoints
for i, row in df2.iterrows():
  row['size'] += sum(df2.loc[df['num_keypoints'] > row['num_keypoints']]['size'])

df2.rename(columns={'size':'num_imgs_with_only_one_ann'},inplace=True)

pd.merge(df,df2)

In [ ]:
## major questions ##
# how will model handle images with more than one person?
# how will model handle images with half a person/person far away? (will num outputted keypoints be constant or dynamic?)